In [17]:
import numpy as np
from scipy.optimize import least_squares
import plotly.graph_objs as go
import plotly.subplots as sp

class RBCModel:
    def __init__(self, params, initial_values):
        self.params = params  # Model parameters
        self.initial_values = initial_values  # Initial values for variables
        self.steady_state_values = None
        self.shocks = {}
        self.model_equations = None

    def declare_model(self, model_type="basic"):
        """Declare the model equations based on the selected model type"""
        if model_type == "basic":
            self.model_equations = self.basic_model
        elif model_type == "log-linear":
            self.model_equations = self.log_linear_model
        else:
            raise ValueError(f"Unknown model type: {model_type}")

    def basic_model(self, variables, params):
        """Basic RBC model equations"""
        c, k, y = variables['c'], variables['k'], variables['y']
        alpha, beta, delta = params['alpha'], params['beta'], params['delta']
        
        # Ensure that capital (k) stays positive
        if k <= 0:
            k = 1e-6  # small positive number to avoid invalid operations

        eq1 = y - k ** alpha * (1 - alpha)  # Cobb-Douglas production function
        eq2 = c - (y - delta * k)  # Consumption equation
        eq3 = k - (1 - delta) * k  # Capital accumulation
        return [eq1, eq2, eq3]

    def log_linear_model(self, variables, params):
        """Log-linearized RBC model equations"""
        c, k, y = variables['c'], variables['k'], variables['y']
        alpha, beta, delta = params['alpha'], params['beta'], params['delta']
        
        # Safeguard to ensure values passed to log() are strictly positive
        c = max(c, 1e-5)
        k = max(k, 1e-5)
        y = max(y, 1e-5)

        # Avoid invalid logarithmic operations by ensuring y - delta * k is positive
        investment_term = y - delta * k
        investment_term = max(investment_term, 1e-5)  # Ensure it is positive

        eq1 = np.log(y) - alpha * np.log(k)  # Log-linearized production function
        eq2 = np.log(c) - np.log(investment_term)  # Log-linearized consumption
        eq3 = np.log(k) - np.log((1 - delta) * k)  # Log-linearized capital accumulation
        return [eq1, eq2, eq3]

    def compute_steady_state(self):
        """Find the steady state by solving the system of equations using least_squares"""
        def steady_state_func(values):
            var_dict = {var: val for var, val in zip(self.variables, values)}
            return self.model_equations(var_dict, self.params)

        # Use better initial guesses based on economic intuition
        initial_guess = [1, 10, 5]  # Improved initial guesses for [c, k, y]
        
        # Using least_squares solver instead of fsolve
        res = least_squares(steady_state_func, initial_guess, max_nfev=5000)
        
        if res.success:
            self.steady_state_values = res.x
        else:
            print(f"Solver failed: {res.message}")
            self.steady_state_values = initial_guess  # Fallback to initial guess
        return self.steady_state_values

    def declare_shocks(self, shock_name, rho, sigma):
        """Define shocks with AR(1) process"""
        self.shocks[shock_name] = {'rho': rho, 'sigma': sigma}

    def simulate_shocks(self, periods=100):
        """Simulate shock series for a given number of periods"""
        shock_series = {}
        for shock, values in self.shocks.items():
            series = np.zeros(periods)
            for t in range(1, periods):
                series[t] = values['rho'] * series[t-1] + np.random.normal(0, values['sigma'])
            shock_series[shock] = series
        return shock_series

    def impulse_response(self, periods=10):
        """Plot impulse response functions for variables"""
        shock_series = self.simulate_shocks(periods)
        shock_name = list(self.shocks.keys())[0]
        shock_values = shock_series[shock_name]

        # Create a Plotly figure for the impulse response
        fig = sp.make_subplots(rows=1, cols=1, subplot_titles=[f'Respuesta ante Impulso: {shock_name}'])

        fig.add_trace(go.Scatter(x=np.arange(periods), y=shock_values, mode='lines', name=f'{shock_name} Shock'), row=1, col=1)

        fig.update_layout(height=400, width=700, title_text="Funciones de Respuesta ante Impulsos (IRFs)")
        fig.show()

    def simulate_endogenous_variables(self, periods=150):
        """Simulate the time series for endogenous variables over multiple periods"""
        shock_series = self.simulate_shocks(periods)
        variables = ['k', 'y', 'c']
        time = np.arange(periods)

        fig = sp.make_subplots(rows=3, cols=1, subplot_titles=[f'Ln{var.capitalize()}' for var in variables])

        for i, var in enumerate(variables):
            simulated_values = np.log(self.initial_values[var]) + np.cumsum(shock_series[list(self.shocks.keys())[0]])
            fig.add_trace(go.Scatter(x=time, y=simulated_values, mode='lines', name=f'{var.capitalize()}'), row=i+1, col=1)

        fig.update_layout(height=900, width=700, title_text="Simulación de Variables Endógenas")
        fig.show()

    def calculate_moments(self, periods=150):
        """Calculate moments for the variables (mean, standard deviation, autocorrelation)"""
        shock_series = self.simulate_shocks(periods)
        variables = ['k', 'y', 'c']

        moments = {}
        for var in variables:
            simulated_values = np.log(self.initial_values[var]) + np.cumsum(shock_series[list(self.shocks.keys())[0]])
            mean = np.mean(simulated_values)
            std_dev = np.std(simulated_values)
            autocorr = np.corrcoef(simulated_values[:-1], simulated_values[1:])[0, 1]

            moments[var] = {'mean': mean, 'std_dev': std_dev, 'autocorrelation': autocorr}

        for var, stats in moments.items():
            print(f"Momentos para {var}:")
            print(f"  Media: {stats['mean']:.4f}")
            print(f"  Desviación Estándar: {stats['std_dev']:.4f}")
            print(f"  Autocorrelación: {stats['autocorrelation']:.4f}")

# Example usage
params = {
    'alpha': 0.46,  # Capital share
    'beta': 0.5,   # Discount factor
    'delta': 0.030, # Depreciation rate
    'rho': 0.4,     # Shock persistence
    'sigma': 0.01   # Standard deviation of shocks
}

initial_values = {
    'k': 10,  # Capital
    'c': 1,   # Consumption
    'y': 4,   # Output
}

# Create the model
model = RBCModel(params, initial_values)

# Choose model type ('basic' or 'log-linear')
model_type = 'log-linear'  # You can change this to 'log-linear'

model.declare_model(model_type)
model.variables = ['c', 'k', 'y']

# Compute steady state
steady_state = model.compute_steady_state()
print("Valores del estado estacionario:", steady_state)

# Define a shock (e.g., technology shock)
model.declare_shocks('tech_shock', rho=0.9, sigma=0.01)

# Plot impulse response
model.impulse_response(periods=50)

# Simulate endogenous variables
model.simulate_endogenous_variables(periods=150)

# Calculate moments
model.calculate_moments(periods=150)



Valores del estado estacionario: [ 3.4371135  21.16983294  4.07220855]


Momentos para k:
  Media: 1.2706
  Desviación Estándar: 0.5702
  Autocorrelación: 0.9993
Momentos para y:
  Media: 0.3543
  Desviación Estándar: 0.5702
  Autocorrelación: 0.9993
Momentos para c:
  Media: -1.0320
  Desviación Estándar: 0.5702
  Autocorrelación: 0.9993
